In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Fungsi untuk memuat gambar dan label
def load_images_from_folder(folder, target_size=(100, 100)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        path = os.path.join(folder, filename)
        img = cv2.imread(path)  # Baca gambar dalam skala abu-abu
        if img is not None:
            img_resized = cv2.resize(img, target_size)  # Resize gambar ke ukuran target
            images.append(img_resized)
            # Menyaring label berdasarkan nama folder
            label = "unknown"
            if "Aluvial" in folder:
                label = "aluvial"
            elif "Vulkanik" in folder:
                label = "vulkanik"
            elif "Humus" in folder:
                label = "humus"
            elif "Latosol" in folder:
                label = "latosol"
            elif "Andosol" in folder:
                label = "andosol"
            labels.append(label)
    return images, labels

# Path ke direktori yang berisi kategori-kategori tanah
base_dir = "drive/MyDrive/tanahore/"

# Memuat gambar dari folder masing-masing kategori
alluvial_images, alluvial_labels = load_images_from_folder(os.path.join(base_dir, "Aluvial"))
vulkanik_images, vulkanik_labels = load_images_from_folder(os.path.join(base_dir, "Vulkanik"))
humus_images, humus_labels = load_images_from_folder(os.path.join(base_dir, "Humus"))
latosol_images, latosol_labels = load_images_from_folder(os.path.join(base_dir, "Latosol"))
andosol_images, andosol_labels = load_images_from_folder(os.path.join(base_dir, "Andosol"))
unknown_images, unknown_labels = load_images_from_folder(os.path.join(base_dir, "Unknown"))

# Menggabungkan gambar dan label dari kategori-kategori yang ada
images = np.concatenate([alluvial_images, vulkanik_images, humus_images, latosol_images, andosol_images])
labels = np.concatenate([alluvial_labels, vulkanik_labels, humus_labels, latosol_labels, andosol_labels, unknown_labels])

# Konversi label menjadi one-hot encoding
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_one_hot = to_categorical(labels_encoded)

# Memisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(images, labels_one_hot, test_size=0.2, random_state=42)

# Membangun arsitektur model CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))  # Jumlah kelas tanah

# Kompilasi model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Melatih model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluasi model pada data uji
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/20
12/12 [==============================] - 7s 507ms/step - loss: 74.8867 - accuracy: 0.2667 - val_loss: 1.5706 - val_accuracy: 0.2667
Epoch 2/20
12/12 [==============================] - 6s 509ms/step - loss: 1.5148 - accuracy: 0.3056 - val_loss: 1.5638 - val_accuracy: 0.3444
Epoch 3/20
12/12 [==============================] - 5s 446ms/step - loss: 1.3256 - accuracy: 0.4861 - val_loss: 1.5341 - val_accuracy: 0.3333
Epoch 4/20
12/12 [==============================] - 7s 596ms/step - loss: 1.2758 - accuracy: 0.6111 - val_loss: 1.3882 - val_accuracy: 0.3778
Epoch 5/20
12/12 [==============================] - 5s 420ms/step - loss: 1.2981 - accuracy: 0.5528 - val_loss: 1.5890 - val_accuracy: 0.4000
Epoch 6/20
12/12 [==============================] - 6s 550ms/step - loss: 1.2878 - accuracy: 0.5306 - val_loss: 1.5486 - val_accuracy: 0.3667
Epoch 7/20
12/12 [==============================] - 6s 456ms/step - loss: 1.1673 - accuracy: 0.5889 - val_loss: 1.6615 - val_accuracy: 0.3667
Epoch